In [6]:
# Parameters

ds =  1# downscale factor for probability smoothing
p_threshold = 0.8# (average) probability threshold

import torch
import alvq
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob as glob
import os

net = torch.load('pix-classifier-alvq-2025-07-09.pt', weights_only=False)

# img = cv2.imread(r"c:\Users\anush\Documents\PostDoc\Croptimal datasets\NAKFielddataset\Spunta\savepatches_demo\12_healthybox_RGBpatch_2.png")
# Path where your images are stored
folder = r"dataset\Spunta"

# Find all files that contain 'RGBpatch' and end with .png (or .jpg, etc.)
files = glob.glob(os.path.join(folder, "**.png"))

print(f"Found {len(files)} images")

# Read all images into a list
for i, f in enumerate(files):
    print(f"image: {f}")
   
    img = cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2HSV) 
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    rgb_feats = (img[..., 0:3].astype(np.float32) * (1/255)).reshape(-1, 3)

    with torch.no_grad():
        scalar = net.probabilities(torch.from_numpy(rgb_feats))

    width = img.shape[1]
    height = img.shape[0]
    # if width==0 or height==0 or img.shape[2]==0:
    #     print(f"Failed to load image: {f}")
    #     continue
    scalar = scalar[..., 1].reshape(img.shape[0:2])
    scalar_orig = scalar
    scalar = cv2.resize(scalar.numpy(), (width // ds, height // ds), interpolation=cv2.INTER_AREA)
    scalar = cv2.resize(scalar, (width, height), interpolation=cv2.INTER_LINEAR)

    # fig, axes = plt.subplots(2, 2, figsize=(10,5))
    # axes[0][0].imshow(img)
    img2 = img.copy()
    img2[scalar < p_threshold, :] = 0
    folder, filename = os.path.split(f)
    name, ext = os.path.splitext(filename)

    # New filename with "_with_seg"
    output_folder = os.path.join(folder, 'HSVFilter')
    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, f"{name}.png")

    cv2.imwrite(output_path, cv2.cvtColor(img2, cv2.COLOR_HSV2BGR))
    print(f"Saved filtered image to: {output_path}")

    

    # mask = (scalar_orig > 0).cpu().numpy().astype(np.float32)  # (H, W) binary mask
    # axes[0][1].imshow(img)
    # axes[1][0].imshow(img2)
    # axes[1][1].imshow(scalar)

Found 57 images
image: dataset\Spunta\2025_5_19_Ziek en gezond2_Spunta_None_1.png
Saved filtered image to: dataset\Spunta\HSVFilter\2025_5_19_Ziek en gezond2_Spunta_None_1.png
image: dataset\Spunta\2025_5_19_Ziek gezond _Spunta_Bladrol_5.png
Saved filtered image to: dataset\Spunta\HSVFilter\2025_5_19_Ziek gezond _Spunta_Bladrol_5.png
image: dataset\Spunta\2025_5_22_Ziek gezond _Spunta_Y-Virus_0.png
Saved filtered image to: dataset\Spunta\HSVFilter\2025_5_22_Ziek gezond _Spunta_Y-Virus_0.png
image: dataset\Spunta\2025_5_22_Ziek gezond _Spunta_Y-Virus_1.png
Saved filtered image to: dataset\Spunta\HSVFilter\2025_5_22_Ziek gezond _Spunta_Y-Virus_1.png
image: dataset\Spunta\2025_5_22_Ziek gezond _Spunta_Y-Virus_2.png
Saved filtered image to: dataset\Spunta\HSVFilter\2025_5_22_Ziek gezond _Spunta_Y-Virus_2.png
image: dataset\Spunta\2025_5_22_Ziek gezond _Spunta_Y-Virus_3.png
Saved filtered image to: dataset\Spunta\HSVFilter\2025_5_22_Ziek gezond _Spunta_Y-Virus_3.png
image: dataset\Spunta\20

In [7]:
import os
import cv2
import random
import numpy as np

def extract_patches(image_path, label_path, save_folder, plantno):
    # Read image
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    
    # Read label file (bounding boxes)
    with open(label_path, 'r') as f:
        lines = f.readlines()
    def save_patch(img_patch, prefix, idx, plantno):
        if len(img_patch.shape) == 3 and img_patch.shape[0]!=0 and img_patch.shape[1]!=0 and img_patch.shape[2]!=0:  # if RGB/BGR
            # img_patch = cv2.cvtColor(img_patch, cv2.COLOR_BGR2GRAY)
            hue_patch = img_patch
        else:
            return
        # Save the patch with the appropriate class prefix
        # hsv_patch = cv2.cvtColor(img_patch, cv2.COLOR_BGR2HSV)  # convert BGR → HSV
        #[:, :, 2]  # extract H channel (hue)

        patch_path = os.path.join(save_folder, f"{plantno}_{prefix}_RGBpatch_{idx}.png")
        cv2.imwrite(patch_path, hue_patch)    
    # Separate healthy (class 0) and unhealthy (class 1) boxes
    healthy_boxes = []
    unhealthy_boxes_Y_virus = []
    unhealthy_boxes_bladroll=[]
    for line in lines:
        

        class_id, x_center, y_center, box_w, box_h = map(float, line.strip().split())
        x_min = int((x_center - box_w / 2) * w)
        y_min = int((y_center - box_h / 2) * h)
        x_max = int((x_center + box_w / 2) * w)
        y_max = int((y_center + box_h / 2) * h)
        
        if int(class_id) == 0:
            healthy_boxes.append((x_min, y_min, x_max, y_max))
        else:
            if "_Y-Virus_" in label_path:
                unhealthy_boxes_Y_virus.append((x_min, y_min, x_max, y_max))
            elif "_Bladrol_" in label_path:    
                unhealthy_boxes_bladroll.append((x_min, y_min, x_max, y_max))

    # Draw bounding boxes on a copy of the image
    image_with_boxes = image.copy()
    for idx, (x_min, y_min, x_max, y_max) in enumerate(healthy_boxes):
        # cv2.rectangle(image_with_boxes, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green for healthy
        region=image_with_boxes[y_min:y_max, x_min:x_max]
        save_patch(region, "healthybox", f"{idx}", f"{plantno}")
        # cv2.putText(image_with_boxes, "Healthy", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

    for idx, (x_min, y_min, x_max, y_max) in enumerate(unhealthy_boxes_Y_virus):
        # cv2.rectangle(image_with_boxes, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Red for unhealthy
        region=image_with_boxes[y_min:y_max, x_min:x_max]
        save_patch(region, "unhealthybox", f"{idx}", f"{plantno}")
        # cv2.putText(image_with_boxes, "Unhealthy", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
    for idx, (x_min, y_min, x_max, y_max) in enumerate(unhealthy_boxes_bladroll):
        # cv2.rectangle(image_with_boxes, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Red for unhealthy
        region=image_with_boxes[y_min:y_max, x_min:x_max]
        save_patch(region, "unhealthy_boxes_bladroll", f"{idx}", f"{plantno}")
    # Display the image
    # Resize image for display
    resized_image = cv2.resize(image_with_boxes, (600, 400))



# Example usage
folder_path= r"dataset\Spunta\HSVFilter"
label_path= r"dataset\Spunta"
save_folder = r"dataset\Spunta\leaf_images"  # Update save location
os.makedirs(save_folder, exist_ok=True)

image_files = [f for f in os.listdir(folder_path) if f.endswith(".png")]
label_files = [f.replace(".png", ".txt") for f in image_files]

for plantno, (img_file, lbl_file) in enumerate(reversed(list(zip(image_files, label_files)))):
    print(img_file)
    img_path = os.path.join(folder_path, img_file)
    lbl_path = os.path.join(label_path, lbl_file)

 
    if os.path.exists(lbl_path):
        extract_patches(img_path, lbl_path, save_folder, plantno)


2025_6_4_Zijlma loods_Spunta_Y-Virus_9.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_7.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_5.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_30.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_3.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_28.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_27.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_25.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_23.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_21.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_17.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_15.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_13.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_11.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_1.png
2025_6_4_Zijlma loods_Spunta_Y-Virus_0.png
2025_6_4_Zijlma loods_Spunta_None_8.png
2025_6_4_Zijlma loods_Spunta_None_6.png
2025_6_4_Zijlma loods_Spunta_None_4.png
2025_6_4_Zijlma loods_Spunta_None_20.png
2025_6_4_Zijlma loods_Spunta_None_2.png
2025_6_4_Zijlma loods_Spunta_None_18.png
2025_6_4_Zijlma loods_Spunta_None_16.png
2025_6_4_Zijlma loo

In [9]:
# Parameters

ds =   2# downscale factor for probability smoothing
p_threshold = 0.9# (average) probability threshold

import torch
import alvq
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob as glob
import os
net = torch.load('pix-classifier-alvq-2025-07-09.pt', weights_only=False)

# img = cv2.imread(r"c:\Users\anush\Documents\PostDoc\Croptimal datasets\NAKFielddataset\Spunta\savepatches_demo\12_healthybox_RGBpatch_2.png")
# Path where your images are stored
folder = r"dataset\Spunta\leaf_images"

# Find all files that contain 'RGBpatch' and end with .png (or .jpg, etc.)
files = glob.glob(os.path.join(folder, "*RGB*.png"))

print(f"Found {len(files)} images")

# Read all images into a list
for i, f in enumerate(files):
    print(f"image: {f}")
   
    img = cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB) 
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    rgb_feats = (img[..., 0:3].astype(np.float32) * (1/255)).reshape(-1, 3)

    with torch.no_grad():
        scalar = net.probabilities(torch.from_numpy(rgb_feats))

    width = img.shape[1]
    height = img.shape[0]
    # if width==0 or height==0 or img.shape[2]==0:
    #     print(f"Failed to load image: {f}")
    #     continue
    scalar = scalar[..., 1].reshape(img.shape[0:2])
    scalar_orig = scalar
    scalar = cv2.resize(scalar.numpy(), (width // ds, height // ds), interpolation=cv2.INTER_AREA)
    scalar = cv2.resize(scalar, (width, height), interpolation=cv2.INTER_LINEAR)

    # fig, axes = plt.subplots(2, 2, figsize=(10,5))
    # axes[0][0].imshow(img)
    img2 = img.copy()
    img2[scalar < p_threshold, :] = 0
    folder, filename = os.path.split(f)
    name, ext = os.path.splitext(filename)

    # New filename with "_with_seg"
    os.makedirs(os.path.join(folder, 'RGBFilter'), exist_ok=True)
    output_path = os.path.join(folder, 'RGBFilter', f"{name}.png")

    # OpenCV expects BGR, so convert back before saving
    cv2.imwrite(output_path, cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))
    print(f"Saved filtered image to: {output_path}")

    

    # mask = (scalar_orig > 0).cpu().numpy().astype(np.float32)  # (H, W) binary mask
    # axes[0][1].imshow(img)
    # axes[1][0].imshow(img2)
    # axes[1][1].imshow(scalar)

Found 3928 images
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_0.png
Saved filtered image to: dataset\Spunta\leaf_images\RGBFilter\0_healthybox_RGBpatch_0.png
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_1.png
Saved filtered image to: dataset\Spunta\leaf_images\RGBFilter\0_healthybox_RGBpatch_1.png
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_10.png
Saved filtered image to: dataset\Spunta\leaf_images\RGBFilter\0_healthybox_RGBpatch_10.png
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_11.png
Saved filtered image to: dataset\Spunta\leaf_images\RGBFilter\0_healthybox_RGBpatch_11.png
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_12.png
Saved filtered image to: dataset\Spunta\leaf_images\RGBFilter\0_healthybox_RGBpatch_12.png
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_13.png
Saved filtered image to: dataset\Spunta\leaf_images\RGBFilter\0_healthybox_RGBpatch_13.png
image: dataset\Spunta\leaf_images\0_healthybox_RGBpatch_14.png

In [11]:
import os
import cv2
import glob

folder = r"dataset\Spunta\leaf_images\RGBFilter"
folder2 = r"dataset\Spunta\leaf_images\RGBFilter\channels_pgm"
os.makedirs(folder2, exist_ok=True)

# Loop through all segmented RGB image
# s
for i, file in enumerate(glob.glob(os.path.join(folder, "*RGBpatch*.png"))):
 
    img = cv2.imread(file)  # BGR in OpenCV
    if img is None:
        print(f"⚠️ Could not read {file}")
        continue
    hsv_patch = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    # Split into channels
    b, g, r = cv2.split(img)  # OpenCV loads as BGR
    h, s, v = cv2.split(hsv_patch)  # OpenCV loads as BGR
    # Build base name
    base = os.path.basename(file)
    name, ext = os.path.splitext(base)

    # Replace "RGBpatch" with single-channel names
    r_name = name.replace("RGBpatch", "Rpatch") + '.pgm'
    g_name = name.replace("RGBpatch", "Gpatch") + '.pgm'
    b_name = name.replace("RGBpatch", "Bpatch") + '.pgm'
    h_name = name.replace("RGBpatch", "Hpatch") + '.pgm'
    s_name = name.replace("RGBpatch", "Spatch") + '.pgm'

    # Save grayscale images
    cv2.imwrite(os.path.join(folder2, r_name), r)
    cv2.imwrite(os.path.join(folder2, g_name), g)
    cv2.imwrite(os.path.join(folder2, b_name), b)
    cv2.imwrite(os.path.join(folder2, h_name), h)
    cv2.imwrite(os.path.join(folder2, s_name), s)

    # print(f"✅ Saved {r_name}, {g_name}, {b_name}")
